In [1]:
# Import required libraries
import pandas as pd
import json
import os
import re
from datetime import datetime
from pathlib import Path
import geopandas as gpd
import folium
import numpy as np

def load_cheapest_rates(prefix: str, output_dir: str = "output") -> tuple[pd.DataFrame, pd.DataFrame, str]:
    """
    Load the latest cheapest rates data for the given prefix.
    
    Args:
        prefix: File prefix to search for (e.g., 'cario_hds_', 'cario_bus_')
        output_dir: Directory to search in (default: 'output')
    
    Returns:
        tuple: (lanes_df, postcodes_df, timestamp)
    """
    def get_latest_files_with_prefix(prefix: str, output_dir: str = "output"):
        """
        Fixed version - Find the latest files with the given prefix in the output directory.
        """
        import os
        import re
        from datetime import datetime
        
        if not os.path.exists(output_dir):
            return None, None, None
        
        latest_ts = None
        latest_raw_file = None
        latest_lanes_file = None
        latest_postcodes_file = None
        
        # Pre-compile regex for speed
        raw_pattern = re.compile(rf'^{re.escape(prefix)}raw_data_(\d{{8}}-\d{{6}})\.csv$')
        lanes_pattern = re.compile(rf'^{re.escape(prefix)}cheapest_per_lane_(\d{{8}}-\d{{6}})\.csv$')
        postcodes_pattern = re.compile(rf'^{re.escape(prefix)}cheapest_per_postcode_(\d{{8}}-\d{{6}})\.csv$')

        files = os.listdir(output_dir)
        print(f"Debug: Found {len(files)} files in {output_dir}")
        
        for fname in files:
            # Check raw data file
            m = raw_pattern.match(fname)
            if m:
                ts_str = m.group(1)
                try:
                    ts = datetime.strptime(ts_str, '%Y%m%d-%H%M%S')
                    if latest_ts is None or ts > latest_ts:
                        latest_ts = ts
                        latest_raw_file = fname
                    print(f"Debug: Found raw file: {fname}, timestamp: {ts_str}")
                except ValueError:
                    continue
            
            # Check lanes file
            m = lanes_pattern.match(fname)
            if m:
                ts_str = m.group(1)
                try:
                    ts = datetime.strptime(ts_str, '%Y%m%d-%H%M%S')
                    if latest_ts is None or ts > latest_ts:
                        latest_ts = ts
                        latest_lanes_file = fname
                    print(f"Debug: Found lanes file: {fname}, timestamp: {ts_str}")
                except ValueError:
                    continue
            
            # Check postcodes file
            m = postcodes_pattern.match(fname)
            if m:
                ts_str = m.group(1)
                try:
                    ts = datetime.strptime(ts_str, '%Y%m%d-%H%M%S')
                    if latest_ts is None or ts > latest_ts:
                        latest_ts = ts
                        latest_postcodes_file = fname
                    print(f"Debug: Found postcodes file: {fname}, timestamp: {ts_str}")
                except ValueError:
                    continue

        print(f"Debug: Latest files found - Raw: {latest_raw_file}, Lanes: {latest_lanes_file}, Postcodes: {latest_postcodes_file}")
        return latest_raw_file, latest_lanes_file, latest_postcodes_file

    raw_file, lanes_file, postcodes_file = get_latest_files_with_prefix(prefix, output_dir)
    
    if not lanes_file or not postcodes_file:
        raise FileNotFoundError(f"No files found with prefix '{prefix}' in {output_dir}/")
    
    # Extract timestamp from filename
    timestamp_match = re.search(rf'{re.escape(prefix)}cheapest_per_lane_(\d{{8}}-\d{{6}})\.csv$', lanes_file)
    if timestamp_match:
        timestamp = timestamp_match.group(1)
    else:
        timestamp = "unknown"
    
    # Load the data
    lanes_df = pd.read_csv(os.path.join(output_dir, lanes_file))
    postcodes_df = pd.read_csv(os.path.join(output_dir, postcodes_file))
    
    print(f"✅ Loaded data from timestamp: {timestamp}")
    print(f"�� Lanes data: {len(lanes_df)} rows")
    print(f"📊 Postcodes data: {len(postcodes_df)} rows")
    
    return lanes_df, postcodes_df, timestamp

def find_cheapest_rates(lanes_df: pd.DataFrame, postcodes_df: pd.DataFrame, 
                       min_price: float = 0, max_price: float = float('inf'),
                       max_transit_days: int = None) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Filter and find the cheapest rates based on criteria.
    
    Args:
        lanes_df: DataFrame with lane-level data
        postcodes_df: DataFrame with postcode-level data
        min_price: Minimum price filter (default: 0)
        max_price: Maximum price filter (default: no limit)
        max_transit_days: Maximum transit days filter (default: no limit)
    
    Returns:
        tuple: (filtered_lanes_df, filtered_postcodes_df)
    """
    # Ensure numeric columns
    lanes_df['freight_price'] = pd.to_numeric(lanes_df['freight_price'], errors='coerce')
    lanes_df['transit_time_days'] = pd.to_numeric(lanes_df['transit_time_days'], errors='coerce')
    postcodes_df['freight_price'] = pd.to_numeric(postcodes_df['freight_price'], errors='coerce')
    postcodes_df['transit_time_days'] = pd.to_numeric(postcodes_df['transit_time_days'], errors='coerce')
    
    # Apply filters
    lanes_filtered = lanes_df[
        (lanes_df['freight_price'] >= min_price) & 
        (lanes_df['freight_price'] <= max_price)
    ].copy()
    
    postcodes_filtered = postcodes_df[
        (postcodes_df['freight_price'] >= min_price) & 
        (postcodes_df['freight_price'] <= max_price)
    ].copy()
    
    if max_transit_days is not None:
        lanes_filtered = lanes_filtered[
            (lanes_filtered['transit_time_days'] <= max_transit_days) | 
            (lanes_filtered['transit_time_days'].isna())
        ]
        postcodes_filtered = postcodes_filtered[
            (postcodes_filtered['transit_time_days'] <= max_transit_days) | 
            (postcodes_filtered['transit_time_days'].isna())
        ]
    
    # Sort by price
    lanes_filtered = lanes_filtered.sort_values('freight_price')
    postcodes_filtered = postcodes_filtered.sort_values('freight_price')
    
    print(f"🔍 Filtered lanes: {len(lanes_filtered)} rows (from {len(lanes_df)})")
    print(f"�� Filtered postcodes: {len(postcodes_filtered)} rows (from {len(postcodes_df)})")
    
    if len(lanes_filtered) > 0:
        print(f"💰 Cheapest lane: ${lanes_filtered.iloc[0]['freight_price']:.2f}")
    if len(postcodes_filtered) > 0:
        print(f"�� Cheapest postcode: ${postcodes_filtered.iloc[0]['freight_price']:.2f}")
    
    return lanes_filtered, postcodes_filtered

def create_choropleth_map(data: pd.DataFrame, price_column: str = 'freight_price', 
                         title: str = 'Freight Rates Map') -> folium.Map:
    """
    Create a choropleth map showing freight rates.
    
    Args:
        data: DataFrame with postcode and price data
        price_column: Column name for price data
        title: Map title
    
    Returns:
        folium.Map: Interactive map
    """
    # Ensure postcode is string and 4 digits
    data = data.copy()
    data['customer_postcode'] = data['customer_postcode'].astype(str).str.zfill(4)
    
    # Clip prices at $400 for better visualization
    data[price_column] = data[price_column].clip(upper=400)
    
    # Load boundary data (you may need to adjust this path)
    try:
        boundary = gpd.read_file("boundary.json")
        boundary['POA_NAME'] = boundary['POA_NAME'].str[:4]
        boundary = boundary[boundary['geometry'].notnull()]
    except FileNotFoundError:
        print("⚠️  boundary.json not found. Creating simple map without boundaries.")
        # Create a simple map without boundaries
        m = folium.Map(location=[-24.15, 133.25], zoom_start=3)
        return m
    
    # Create base map
    m = folium.Map(location=[-24.15, 133.25], zoom_start=3)
    
    # Add choropleth layer
    folium.Choropleth(
        geo_data=boundary,
        data=data,
        bins=8,
        columns=["customer_postcode", price_column],
        key_on="feature.properties.POA_NAME",
        fill_color="OrRd",
        nan_fill_color="lightgray",
        fill_opacity=0.7,
        line_opacity=0.2,
        name='Freight Price',
        highlight=True,
        legend_name=f"{title} ($AUD)",
    ).add_to(m)
    
    # Add layer control
    folium.LayerControl().add_to(m)
    
    return m

# One-stop analysis function
def analyze_cario_rates(prefix: str = "cario_hds_", 
                       min_price: float = 0, 
                       max_price: float = 300,
                       max_transit_days: int = 7) -> tuple[pd.DataFrame, pd.DataFrame, folium.Map]:
    """
    Complete analysis function that loads, filters, and visualizes Cario rates.
    
    Args:
        prefix: File prefix to search for
        min_price: Minimum price filter
        max_price: Maximum price filter
        max_transit_days: Maximum transit days filter
    
    Returns:
        tuple: (lanes_df, postcodes_df, map)
    """
    print(f"🔍 Loading data with prefix: {prefix}")
    
    # Load data
    lanes_df, postcodes_df, timestamp = load_cheapest_rates(prefix)
    
    # Filter data
    lanes_filtered, postcodes_filtered = find_cheapest_rates(
        lanes_df, postcodes_df, min_price, max_price, max_transit_days
    )
    
    # Create map
    map_title = f"{prefix.replace('_', ' ').title()} Freight Rates"
    m = create_choropleth_map(postcodes_filtered, title=map_title)
    
    return lanes_filtered, postcodes_filtered, m

# Test the fixed function
prefix = "cario_hds_"

# Load HDS rates with filters
lanes, postcodes, map = analyze_cario_rates("cario_hds_", max_price=200, max_transit_days=5)
map

🔍 Loading data with prefix: cario_hds_
Debug: Found 13 files in output
Debug: Found raw file: cario_hds_raw_data_20250903-144903.csv, timestamp: 20250903-144903
Debug: Found raw file: cario_hds_raw_data_20250903-143514.csv, timestamp: 20250903-143514
Debug: Found lanes file: cario_hds_cheapest_per_lane_20250903-144903.csv, timestamp: 20250903-144903
Debug: Found lanes file: cario_hds_cheapest_per_lane_20250903-143514.csv, timestamp: 20250903-143514
Debug: Found postcodes file: cario_hds_cheapest_per_postcode_20250903-144903.csv, timestamp: 20250903-144903
Debug: Found postcodes file: cario_hds_cheapest_per_postcode_20250903-143514.csv, timestamp: 20250903-143514
Debug: Latest files found - Raw: cario_hds_raw_data_20250903-144903.csv, Lanes: None, Postcodes: None


FileNotFoundError: No files found with prefix 'cario_hds_' in output/

In [2]:
# Import required libraries
import pandas as pd
import json
import os
import re
from datetime import datetime
from pathlib import Path
import geopandas as gpd
import folium
import numpy as np

def load_cheapest_rates(prefix: str, output_dir: str = "output") -> tuple[pd.DataFrame, pd.DataFrame, str]:
    """
    Load the latest cheapest rates data for the given prefix.
    
    Args:
        prefix: File prefix to search for (e.g., 'cario_hds_', 'cario_bus_')
        output_dir: Directory to search in (default: 'output')
    
    Returns:
        tuple: (lanes_df, postcodes_df, timestamp)
    """
    def get_latest_files_with_prefix(prefix: str, output_dir: str = "output"):
        """
        Fixed version - Find the latest files with the given prefix in the output directory.
        """
        import os
        import re
        from datetime import datetime
        
        if not os.path.exists(output_dir):
            return None, None, None
        
        latest_ts = None
        latest_raw_file = None
        latest_lanes_file = None
        latest_postcodes_file = None
        
        # Pre-compile regex for speed
        raw_pattern = re.compile(rf'^{re.escape(prefix)}raw_data_(\d{{8}}-\d{{6}})\.csv$')
        lanes_pattern = re.compile(rf'^{re.escape(prefix)}cheapest_per_lane_(\d{{8}}-\d{{6}})\.csv$')
        postcodes_pattern = re.compile(rf'^{re.escape(prefix)}cheapest_per_postcode_(\d{{8}}-\d{{6}})\.csv$')

        files = os.listdir(output_dir)
        print(f"Debug: Found {len(files)} files in {output_dir}")
        
        for fname in files:
            # Check raw data file
            m = raw_pattern.match(fname)
            if m:
                ts_str = m.group(1)
                try:
                    ts = datetime.strptime(ts_str, '%Y%m%d-%H%M%S')
                    if latest_ts is None or ts >= latest_ts:  # Changed > to >=
                        latest_ts = ts
                        latest_raw_file = fname
                    print(f"Debug: Found raw file: {fname}, timestamp: {ts_str}")
                except ValueError:
                    continue
            
            # Check lanes file
            m = lanes_pattern.match(fname)
            if m:
                ts_str = m.group(1)
                try:
                    ts = datetime.strptime(ts_str, '%Y%m%d-%H%M%S')
                    if latest_ts is None or ts >= latest_ts:  # Changed > to >=
                        latest_ts = ts
                        latest_lanes_file = fname
                    print(f"Debug: Found lanes file: {fname}, timestamp: {ts_str}")
                except ValueError:
                    continue
            
            # Check postcodes file
            m = postcodes_pattern.match(fname)
            if m:
                ts_str = m.group(1)
                try:
                    ts = datetime.strptime(ts_str, '%Y%m%d-%H%M%S')
                    if latest_ts is None or ts >= latest_ts:  # Changed > to >=
                        latest_ts = ts
                        latest_postcodes_file = fname
                    print(f"Debug: Found postcodes file: {fname}, timestamp: {ts_str}")
                except ValueError:
                    continue

        print(f"Debug: Latest files found - Raw: {latest_raw_file}, Lanes: {latest_lanes_file}, Postcodes: {latest_postcodes_file}")
        return latest_raw_file, latest_lanes_file, latest_postcodes_file

    raw_file, lanes_file, postcodes_file = get_latest_files_with_prefix(prefix, output_dir)
    
    if not lanes_file or not postcodes_file:
        raise FileNotFoundError(f"No files found with prefix '{prefix}' in {output_dir}/")
    
    # Extract timestamp from filename
    timestamp_match = re.search(rf'{re.escape(prefix)}cheapest_per_lane_(\d{{8}}-\d{{6}})\.csv$', lanes_file)
    if timestamp_match:
        timestamp = timestamp_match.group(1)
    else:
        timestamp = "unknown"
    
    # Load the data
    lanes_df = pd.read_csv(os.path.join(output_dir, lanes_file))
    postcodes_df = pd.read_csv(os.path.join(output_dir, postcodes_file))
    
    print(f"✅ Loaded data from timestamp: {timestamp}")
    print(f" Lanes data: {len(lanes_df)} rows")
    print(f"📊 Postcodes data: {len(postcodes_df)} rows")
    
    return lanes_df, postcodes_df, timestamp

def find_cheapest_rates(lanes_df: pd.DataFrame, postcodes_df: pd.DataFrame, 
                       min_price: float = 0, max_price: float = float('inf'),
                       max_transit_days: int = None) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Filter and find the cheapest rates based on criteria.
    
    Args:
        lanes_df: DataFrame with lane-level data
        postcodes_df: DataFrame with postcode-level data
        min_price: Minimum price filter (default: 0)
        max_price: Maximum price filter (default: no limit)
        max_transit_days: Maximum transit days filter (default: no limit)
    
    Returns:
        tuple: (filtered_lanes_df, filtered_postcodes_df)
    """
    # Ensure numeric columns
    lanes_df['freight_price'] = pd.to_numeric(lanes_df['freight_price'], errors='coerce')
    lanes_df['transit_time_days'] = pd.to_numeric(lanes_df['transit_time_days'], errors='coerce')
    postcodes_df['freight_price'] = pd.to_numeric(postcodes_df['freight_price'], errors='coerce')
    postcodes_df['transit_time_days'] = pd.to_numeric(postcodes_df['transit_time_days'], errors='coerce')
    
    # Apply filters
    lanes_filtered = lanes_df[
        (lanes_df['freight_price'] >= min_price) & 
        (lanes_df['freight_price'] <= max_price)
    ].copy()
    
    postcodes_filtered = postcodes_df[
        (postcodes_df['freight_price'] >= min_price) & 
        (postcodes_df['freight_price'] <= max_price)
    ].copy()
    
    if max_transit_days is not None:
        lanes_filtered = lanes_filtered[
            (lanes_filtered['transit_time_days'] <= max_transit_days) | 
            (lanes_filtered['transit_time_days'].isna())
        ]
        postcodes_filtered = postcodes_filtered[
            (postcodes_filtered['transit_time_days'] <= max_transit_days) | 
            (postcodes_filtered['transit_time_days'].isna())
        ]
    
    # Sort by price
    lanes_filtered = lanes_filtered.sort_values('freight_price')
    postcodes_filtered = postcodes_filtered.sort_values('freight_price')
    
    print(f"🔍 Filtered lanes: {len(lanes_filtered)} rows (from {len(lanes_df)})")
    print(f" Filtered postcodes: {len(postcodes_filtered)} rows (from {len(postcodes_df)})")
    
    if len(lanes_filtered) > 0:
        print(f"💰 Cheapest lane: ${lanes_filtered.iloc[0]['freight_price']:.2f}")
    if len(postcodes_filtered) > 0:
        print(f" Cheapest postcode: ${postcodes_filtered.iloc[0]['freight_price']:.2f}")
    
    return lanes_filtered, postcodes_filtered

def create_choropleth_map(data: pd.DataFrame, price_column: str = 'freight_price', 
                         title: str = 'Freight Rates Map') -> folium.Map:
    """
    Create a choropleth map showing freight rates.
    
    Args:
        data: DataFrame with postcode and price data
        price_column: Column name for price data
        title: Map title
    
    Returns:
        folium.Map: Interactive map
    """
    # Ensure postcode is string and 4 digits
    data = data.copy()
    data['customer_postcode'] = data['customer_postcode'].astype(str).str.zfill(4)
    
    # Clip prices at $400 for better visualization
    data[price_column] = data[price_column].clip(upper=400)
    
    # Load boundary data (you may need to adjust this path)
    try:
        boundary = gpd.read_file("boundary.json")
        boundary['POA_NAME'] = boundary['POA_NAME'].str[:4]
        boundary = boundary[boundary['geometry'].notnull()]
    except FileNotFoundError:
        print("⚠️  boundary.json not found. Creating simple map without boundaries.")
        # Create a simple map without boundaries
        m = folium.Map(location=[-24.15, 133.25], zoom_start=3)
        return m
    
    # Create base map
    m = folium.Map(location=[-24.15, 133.25], zoom_start=3)
    
    # Add choropleth layer
    folium.Choropleth(
        geo_data=boundary,
        data=data,
        bins=8,
        columns=["customer_postcode", price_column],
        key_on="feature.properties.POA_NAME",
        fill_color="OrRd",
        nan_fill_color="lightgray",
        fill_opacity=0.7,
        line_opacity=0.2,
        name='Freight Price',
        highlight=True,
        legend_name=f"{title} ($AUD)",
    ).add_to(m)
    
    # Add layer control
    folium.LayerControl().add_to(m)
    
    return m

# One-stop analysis function
def analyze_cario_rates(prefix: str = "cario_hds_", 
                       min_price: float = 0, 
                       max_price: float = 300,
                       max_transit_days: int = 7) -> tuple[pd.DataFrame, pd.DataFrame, folium.Map]:
    """
    Complete analysis function that loads, filters, and visualizes Cario rates.
    
    Args:
        prefix: File prefix to search for
        min_price: Minimum price filter
        max_price: Maximum price filter
        max_transit_days: Maximum transit days filter
    
    Returns:
        tuple: (lanes_df, postcodes_df, map)
    """
    print(f"🔍 Loading data with prefix: {prefix}")
    
    # Load data
    lanes_df, postcodes_df, timestamp = load_cheapest_rates(prefix)
    
    # Filter data
    lanes_filtered, postcodes_filtered = find_cheapest_rates(
        lanes_df, postcodes_df, min_price, max_price, max_transit_days
    )
    
    # Create map
    map_title = f"{prefix.replace('_', ' ').title()} Freight Rates"
    m = create_choropleth_map(postcodes_filtered, title=map_title)
    
    return lanes_filtered, postcodes_filtered, m

# Test the fixed function
prefix = "cario_hds_"

# Load HDS rates with filters
lanes, postcodes, map = analyze_cario_rates("cario_hds_", max_price=200, max_transit_days=5)
map

🔍 Loading data with prefix: cario_hds_
Debug: Found 13 files in output
Debug: Found raw file: cario_hds_raw_data_20250903-144903.csv, timestamp: 20250903-144903
Debug: Found raw file: cario_hds_raw_data_20250903-143514.csv, timestamp: 20250903-143514
Debug: Found lanes file: cario_hds_cheapest_per_lane_20250903-144903.csv, timestamp: 20250903-144903
Debug: Found lanes file: cario_hds_cheapest_per_lane_20250903-143514.csv, timestamp: 20250903-143514
Debug: Found postcodes file: cario_hds_cheapest_per_postcode_20250903-144903.csv, timestamp: 20250903-144903
Debug: Found postcodes file: cario_hds_cheapest_per_postcode_20250903-143514.csv, timestamp: 20250903-143514
Debug: Latest files found - Raw: cario_hds_raw_data_20250903-144903.csv, Lanes: cario_hds_cheapest_per_lane_20250903-144903.csv, Postcodes: cario_hds_cheapest_per_postcode_20250903-144903.csv
✅ Loaded data from timestamp: 20250903-144903
 Lanes data: 3 rows
📊 Postcodes data: 3 rows
🔍 Filtered lanes: 2 rows (from 3)
 Filtered po

DriverError: boundary.json: No such file or directory